In [1]:
#set up a mongoDB NoSQL database
from pymongo import MongoClient

client = MongoClient("localhost",27017)
db = client['user_password_db']
users = db['users']

In [2]:
# Insert some user/password pairs into the database
users.insert_one({'username': 'Admin', 'password': '1234'})
users.insert_one({'username': 'user1', 'password': 'password1'})
users.insert_one({'username': 'user2', 'password': 'password2'})

In [3]:
#imports for flask
from flask import Flask, render_template, redirect, url_for
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField # to create respective fields in the login.html
from wtforms.validators import DataRequired #to check if user input is empty


In [4]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret_key'
app.config['WTF_CSRF_ENABLED'] = False

In [5]:
#define fields for the login.html
class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Submit')

In [6]:
#handles redirection to initial login.html page 
#aswell as redirection to index.html if both username and password are entered and the submit button is pressed
import json
@app.route('/', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        print(f"The user entered the following data: {form.data}")
        #check if the user is stored in the mongoDB database:
        user_entered = users.find_one({'username': form.username.data})
        if user_entered:
            print(f"the user was found in the database: {user_entered}")
            password_entered = form.password.data
            #if user_entered['password'] == password_entered: 
            #the previous line is safe since the comparison checks for data types and: string!=dictionary
            
            results = users.find_one({'username': form.username.data, 'password': password_entered})
            if results:
                print(f"The password was correct. The user entered: {password_entered}")
                print(f"It matches the database password: {user_entered['password']}")
                return redirect(url_for('index')) 
            else:
                #try again, allowing code
                condition = eval(password_entered)  #allows code to be in here
                results = users.find_one({'username': form.username.data, 'password': condition})
                if results:
                    print(f"The password was accepted. The user entered: {password_entered}")
                    print(f"It matches the database password: {user_entered['password']}")
                    return redirect(url_for('index'))
                else:    
                    print(f"The password was incorrect: {password_entered}")
                    print(f"It should have been: {user_entered['password']}")
                    return render_template('login.html', form=form)
        else:
            print(f"the user was not found in the database: {user_entered}")
            return render_template('login.html', form=form)
    else:
        # print form data and errors for debugging
        print(f"The user entered the following data: {form.data}")
        print("Note that the first call is handled as if the user submits no data")
        print(form.errors)
    return render_template('login.html', form=form)

In [7]:
#redirects to index.html and defines the content of it
@app.route('/index')
def index():
    user = 'Admin'
    posts = [{'author': {'username': 'User 1'}, 'body': 'Post 1'},
        {'author': {'username': 'User 2'}, 'body': 'Post 2'}]
    return render_template('index.html', user=user, posts=posts)

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Jun/2023 15:49:10] "GET / HTTP/1.1" 200 -


The user entered the following data: {'username': None, 'password': None, 'submit': False}
Note that the first call is handled as if the user submits no data
{}


127.0.0.1 - - [11/Jun/2023 15:49:14] "POST / HTTP/1.1" 200 -


The user entered the following data: {'username': 'Admin', 'password': '1', 'submit': True}
the user was found in the database: {'_id': ObjectId('6485b537383e71eac3747df1'), 'username': 'Admin', 'password': '1234'}
The password was incorrect: 1
It should have been: 1234


127.0.0.1 - - [11/Jun/2023 15:49:15] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [11/Jun/2023 15:49:15] "GET /index HTTP/1.1" 200 -


The user entered the following data: {'username': 'Admin', 'password': '1234', 'submit': True}
the user was found in the database: {'_id': ObjectId('6485b537383e71eac3747df1'), 'username': 'Admin', 'password': '1234'}
The password was correct. The user entered: 1234
It matches the database password: 1234


127.0.0.1 - - [11/Jun/2023 15:49:27] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [11/Jun/2023 15:49:27] "GET /index HTTP/1.1" 200 -


The user entered the following data: {'username': 'Admin', 'password': "{'$ne': ''}", 'submit': True}
the user was found in the database: {'_id': ObjectId('6485b537383e71eac3747df1'), 'username': 'Admin', 'password': '1234'}
The password was accepted. The user entered: {'$ne': ''}
It matches the database password: 1234


In [ ]:
# use {'$ne': ''}